In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB

col = ['age', 'workclass', 'final_weight', 'education', 'education-num', 'marital_status', 'occupation', 'relationship', 
       'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']
df_train = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', names=col)
df_test = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test', names=col,skiprows=1)

#print(df_train.head())
#print(df_test.head())
#print(df_train.info())
#print(df_test.info())
#print(df_train.describe())
#print(df_test.describe())

#print(df_train.apply(lambda x: x.isnull().sum()))
#print(df_test.apply(lambda x: x.isnull().sum()))

df_train = df_train[df_train.apply(lambda x:x != ' ?', axis=1)].dropna().reindex()
df_test = df_test[df_test.apply(lambda x:x != ' ?', axis=1)].dropna().reindex()

cat_columns = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'income']
numeric_columns = ['age', 'final_weight', 'capital_gain', 'capital_loss']

#X_train = df_train.drop(['income'], axis=1)
#X_test = df_train.drop(['income'], axis=1)
#scaler = StandardScaler()
#df_train = scaler.fit_transform(df_train)
#df_test = scaler.fit_transform(df_test)

#print(df_train.describe())

replace_map = {}
for columns in cat_columns:
    labels = df_train[columns].astype('category').cat.categories.tolist()
    replace = {columns: {k:v for k,v in zip(labels, list(range(0,len(labels))))}}
    replace_map.update(replace)

df_train['sex_code'] = np.where(df_train['sex'].str.contains('Female'), 0, 1)
df_train['income_code'] = np.where(df_train['income'].str.contains('>50K'), 1, 0)
df_test['sex_code'] = np.where(df_test['sex'].str.contains('Female'), 0, 1)
df_test['income_code'] = np.where(df_test['income'].str.contains('>50K'), 1, 0)

dummy_columns = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'native_country']
#dummy_columns = ['workclass', 'education', 'occupation', 'relationship', 'race', 'native_country']

df_train = pd.get_dummies(df_train, columns=dummy_columns)
df_test = pd.get_dummies(df_test, columns=dummy_columns)


#sns.pairplot(df_train[numeric_columns])
#plt.show()

df_train['final_weight_log'] = np.log(df_train['final_weight'])
df_train['capital_gain_log'] = np.log(df_train['capital_gain'] + 0.0001)
df_train['capital_loss_log'] = np.log(df_train['capital_loss'] + 0.0001)

df_test['final_weight_log'] = np.log(df_test['final_weight'])
df_test['capital_gain_log'] = np.log(df_test['capital_gain'] + 0.0001)
df_test['capital_loss_log'] = np.log(df_test['capital_loss'] + 0.0001)

X_train = df_train.drop(['final_weight','education-num', 'sex', 'capital_gain', 'capital_loss', \
                         'income'], axis=1)
y_train = df_train['income_code']
X_test = df_test.drop(['final_weight', 'education-num', 'sex', 'capital_gain', 'capital_loss', \
                       'income'],axis=1)
X_test['native_country_ Holand-Netherlands'] = 0
y_test = df_test['income_code']

#print(X_train.columns.difference(X_test.columns))

lr = LogisticRegression()
lr.fit(X_train, y_train)
print(lr.score(X_test, y_test))

nb = GaussianNB()
nb.fit(X_train, y_train)
print(nb.score(X_test, y_test))

0.27702523240371846
1.0
